In [1]:
from IPython.display import display
import sympy as sym
from sympy.assumptions.assume import global_assumptions
import time
import pandas as pd
import cpuinfo
sym.init_printing(use_latex='mathjax')
CARTESIAN = 0
CYLINDRICAL = 1
SPHERICAL = 2
#Начальная система
r = sym.Symbol('r', real=True, positive=True)
phi = sym.Symbol('phi', real=True, nonnegative=True)
theta = sym.Symbol('theta', real=True)
x = sym.Symbol('x', real=True)
y = sym.Symbol('y', real=True)
z = sym.Symbol('z', real=True)
global_assumptions.add(sym.Q.positive(sym.cos(theta)), sym.Q.positive(r))
#Конечная система
R = sym.Symbol('R', real=True, positive=True)
Phi = sym.Symbol('Phi', real=True)
Theta = sym.Symbol('Theta', real=True)
X = sym.Symbol('X', real=True)
Y = sym.Symbol('Y', real=True)
Z = sym.Symbol('Z', real=True)
global_assumptions.add(sym.Q.positive(sym.cos(Theta)), sym.Q.positive(R))
I1 = sym.Symbol('I1', real=True)
I2 = sym.Symbol('I2', real=True)
I3 = sym.Symbol('I3', real=True)
mu = sym.Symbol('mu', real=True, positive=True)
global_assumptions.add(sym.Q.positive(mu))
P = sym.Function('P', real=True, positive=True)(r)
Psi = sym.Symbol('psi', real=True)
k = sym.Symbol('k', real=True, positive=True)
global_assumptions.add(sym.Q.positive(P), sym.Q.positive(sym.diff(P, r)), sym.Q.positive(k))

In [2]:
def BlatzAndKoSimple():
    '''
    Возвращает выражение энергии упрощённого варианта материала Блейтца и Ко.
    '''
    return sym.Rational(1, 2) * mu * (I2 / I3 + 2 * sym.sqrt(I3) - 5)

In [3]:
def BlatzAndKoHypothetical():
    '''
    Возвращает выражение энергии гипотетического варианта материала Блейтца и Ко.
    '''
    alpha = sym.Symbol('alpha', real=True)
    return sym.Rational(1, 2) * mu * (I1 + 1 / alpha*(sym.Pow(I3, -alpha) - 1) - 3)

In [4]:
def BlatzAndKoFull():
    '''
    Возвращает выражение энергии полного варианта материала Блейтца и Ко.
    '''
    alpha = sym.Symbol('alpha', real=True)
    beta = sym.Symbol('beta', real=True)
    W = sym.Rational(1, 2) * mu * beta * (I1 + 1 / alpha*(sym.Pow(I3, -alpha) - 1) - 3)
    W = W + sym.Rational(1, 2) * mu * (1 - beta) * (I2 * sym.Pow(I3, -1) + 1 / alpha * (sym.Pow(I3, alpha) - 1) - 3)
    return W

In [5]:
def MurnaghanMaterial():
    '''
    Возвращает выражение энергии материала Мурнагана.
    '''
    lambdas = sym.Symbol('lambda', real=True)
    m = sym.Symbol('m', real=True)
    l = sym.Symbol('l', real=True)
    n = sym.Symbol('n', real=True)
    W = (-3 * lambdas - 2 * mu + sym.Rational(9, 2) * l + n / 2) * I1
    W = W + sym.Rational(1, 2) * (lambdas + 2 * mu - 3 * l - 2 * m) * sym.Pow(I1, 2)
    W = W + (-2 * mu + 3 * m - n / 2) * I2
    W = W - m * I1 * I2
    W = W + sym.Rational(1, 6) * (l + 2*m) * sym.Pow(I1, 3)
    W = W + n / 2*(I3 - 1)
    return sym.Rational(1, 4) * W

In [6]:
def SignoriniBody():
    '''
    Возвращает выражение энергии тела Синьорини.
    '''
    lambdas = sym.Symbol('lambda', real=True)
    return (sym.Rational(4, 9) * mu * ((I2 / I3 + 3)*I2 / I3 - 18*sym.sqrt(1 / I3)) + (9 * lambdas + 5 * mu) * sym.Pow((sym.Rational(1, 3) * I2 / I3 - 1), 2)) / 8 * sym.sqrt(I3)

In [7]:
def GetI1(G):
    '''
    Возвращает выражение инварианта I_1 тензора второго ранга G.
    '''
    return sym.simplify(sym.Trace(G))

In [8]:
def GetI2(G):
    '''
    Возвращает выражение инварианта I_2 тензора второго ранга G.
    '''
    return sym.simplify(sym.Rational(1, 2) * (sym.Pow(sym.Trace(G), 2) - sym.Trace(G * G)))

In [9]:
def GetI3(G):
    '''
    Возвращает выражение инварианта I_3 тензора второго ранга G.
    '''
    return sym.simplify(sym.det(G))

In [10]:
def PiolaTensorSimple(G, C):
    '''
    Возвращает тензор Пиола, посчитанный по формуле для упрощённого варианта материала Блейтца и Ко.
    '''
    PT = mu / I3 * (sym.eye(3) * I1 - G + (sym.Pow(I3, 1.5) - I2) * G.inv()) * C
    PT = MySimplifyMatrix(PT.subs([(I1, GetI1(G)), (I2, GetI2(G)), (I3, GetI3(G))]))
    return PT

In [11]:
def MySimplifyMatrix(A):
    '''
    Упрощение элементов матрицы или вектора с использованием функции MySimplify.
    '''
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            A[i, j] = MySimplify(A[i, j])
    return A

In [12]:
def MySimplify(A):
    '''
    Упрощение символьных выражений с учётом глобальных условий и правила перемножения степеней.
    '''
    return sym.simplify(sym.refine(sym.powdenest(A, force=True)), rational=True)

In [13]:
def dW(W, G):
    '''
    Выражение производной выражения W по тензору второго ранга G.
    '''
    E = sym.eye(3)
    I1G = E
    I2G = I1 * E - G
    I3G = I3 * G.inv()
    dWdG = sym.diff(W, I1) * I1G + sym.diff(W, I2) * I2G + sym.diff(W, I3) * I3G
    dWdG = MySimplifyMatrix(dWdG.subs([(I1, GetI1(G)), (I2, GetI2(G)), (I3, GetI3(G))]))
    return dWdG

In [14]:
def VectorGradient(B, Q, Gamma, H):
    '''
    Выражение градиента вектора B, приведённое к ортонормированному базису.
    Q - символы криволинейных координат.
    Gamma - символы Кристоффеля второго рода.
    H - коэффициенты Ламе.
    '''
    #Приведено к ортонормированному базису.
    NablaA = sym.zeros(3, 3)
    if len(B.shape) == 2:
        A = B[0, :]
    else:
        A = B
    for N in range(0, 3):
        for K in range(0, 3):
            NablaA[K, N] = sym.simplify((sym.diff(A[N], Q[K]) + sum(A[M] * Gamma[N, M, K] for M in range(3))) / H[K] * H[N])
    return NablaA

In [15]:
def TensorSecondRankCov(P, Q, Gamma, M, N, S):
    '''
    Выражение ковариантной производной компоненты тензора второго ранга P.
    Q - символы криволинейных координат.
    Gamma - символы Кристоффеля второго рода.
    M, N, S - индексы.
    '''
    #Формула 38.6.
    sm = sym.diff(P[M, N], Q[S])
    sm += sum(Gamma[M, T, S] * P[T, N] for T in range(3))
    sm += sum(Gamma[N, T, S] * P[M, T] for T in range(3))
    return MySimplify(sm)

In [16]:
def TensorSecondRankDiv(P, Q, Gamma, H):
    '''
    Выражение дивергенции тензора второго ранга P, приведённое к ортонормированному базису.
    Q - символы криволинейных координат.
    Gamma - символы Кристоффеля второго рода.
    H - коэффициенты Ламе.
    '''
    #Приведено к ортонормированному базису.
    DivP = sym.zeros(1, 3)
    for N in range(3):
        DivP[N] = MySimplify(sum(TensorSecondRankCov(P, Q, Gamma, M, N, M) for M in range(3)) * H[N])
    return DivP

In [17]:
def dotprod(a, b):
    '''
    Скалярное произведение векторов a и b.
    '''
    return sym.simplify(sum(a[i] * b[i] for i in range(3)))

In [18]:
def Reference_configuration(coordType):
    '''
    Вычисление характеристик отсчётной конфигурации.
    coordType - тип системы координат (0 - прямоугольная, 1 - цилиндрическая, 2 - сферическая).
    '''
    x_ = sym.zeros(1, 3) #Уравнения отсчётной конфигурации.
    rt = sym.zeros(1, 3) #Радиус-вектор отсчётной конфигурации.
    q = sym.zeros(1, 3)
    if coordType == CARTESIAN:
            q[0] = x
            q[1] = y
            q[2] = z
            rt[0] = x
            rt[1] = y
            rt[2] = z
            x_[0] = x
            x_[1] = y
            x_[2] = z
            #print("Начальная система координат: прямоугольная.")
    elif coordType == CYLINDRICAL:
            x_[0] = r * sym.cos(phi)
            x_[1] = r * sym.sin(phi)
            x_[2] = z
            rt[0] = r
            rt[1] = 0
            rt[2] = z
            q[0] = r
            q[1] = phi
            q[2] = z
            #print("Начальная система координат: цилиндрическая.")
    else:
            x_[0] = r * sym.cos(theta) * sym.cos(phi)
            x_[1] = r * sym.cos(theta) * sym.sin(phi)
            x_[2] = r * sym.sin(theta)
            rt[0] = r
            rt[1] = 0
            rt[2] = 0
            q[0] = r
            q[1] = phi
            q[2] = theta
            #print("Начальная система координат: сферическая.")
    r_ = sym.zeros(3, 3)
    for n in range(0, 3):
        for s in range(0, 3):
            r_[n, s] = sym.diff(x_[s], q[n])
    #print("Основной базис:")
    #sym.pprint(r_)
    g_ = sym.zeros(3, 3)
    for m in range(0, 3):
        for n in range(0, 3):
            for i in range(0, 3):
                g_[m, n] = sym.simplify(g_[m, n]+r_[m, i]*r_[n, i])
    #print("Метрические коэффициенты:")
    #sym.pprint(g_)
    g = sym.Inverse(g_)
    #print("Обратная матрица метрических коэффициентов: ")
    #sym.pprint(g)
    h = sym.zeros(1, 3)
    for m in range(0, 3):
        #refine - упрощение выражений с использованием предположений. Предположения могут быть или в global_assumptions, или могут быть переданы напрямую при вызове функции.
        #powdenest - упрощение выражения со степенями, используя правило перемножения степеней при возведении степени в степень.
        h[m] = MySimplify(sym.sqrt(g_[m, m]))
    #print("Коэффициенты Ламе:")
    #sym.pprint(h)
    gamma = sym.MutableDenseNDimArray.zeros(3, 3, 3)
    for n in range(0, 3):
        for k in range(0, 3):
            for m in range(0, 3):
                gamma[m, n, k] = sym.simplify(1/2*sum(g[m, s]*(sym.diff(g_[n, s], q[k])+sym.diff(g_[k, s], q[n])-sym.diff(g_[n, k], q[s]))
                for s in range(3)))
    #print("Символы Кристоффеля второго рода:")
    #sym.pprint(gamma)
    return q, rt, r_, h, gamma

In [19]:
def Actual_configuration(coordType):
    '''
    Вычисление характеристик текущей конфигурации.
    coordType - тип системы координат (0 - прямоугольная, 1 - цилиндрическая, 2 - сферическая).
    '''
    X_ = sym.zeros(1, 3) #Уравнения конечной системы.
    Rt = sym.zeros(1, 3) #Радиус-вектор конечной системы.
    Q = sym.zeros(1, 3)
    if coordType == CARTESIAN:
            Q[0] = X
            Q[1] = Y
            Q[2] = Z
            Rt[0] = X
            Rt[1] = Y
            Rt[2] = Z
            X_[0] = X
            X_[1] = Y
            X_[2] = Z
            #print("Конечная система координат: прямоугольная.")
    elif coordType == CYLINDRICAL:
            X_[0] = R * sym.cos(Phi)
            X_[1] = R * sym.sin(Phi)
            X_[2] = Z
            Rt[0] = R
            Rt[1] = 0
            Rt[2] = Z
            Q[0] = R
            Q[1] = Phi
            Q[2] = Z
            #print("Конечная система координат: цилиндрическая.")
    else:
            X_[0] = R * sym.cos(Theta) * sym.cos(Phi)
            X_[1] = R * sym.cos(Theta) * sym.sin(Phi)
            X_[2] = R * sym.sin(Theta)
            Rt[0] = R
            Rt[1] = 0
            Rt[2] = 0
            Q[0] = R
            Q[1] = Phi
            Q[2] = Theta
            #print("Конечная система координат: сферическая.")
    R_ = sym.zeros(3, 3)
    for n in range(0, 3):
        for s in range(0, 3):
            R_[n, s] = sym.diff(X_[s], Q[n])
    #print("Основной базис:")
    #sym.pprint(R_)
    G_ = sym.zeros(3, 3)
    for M in range(0, 3):
        for N in range(0, 3):
            for I in range(0, 3):
                G_[M, N] = sym.simplify(G_[M, N] + R_[M, I] * R_[N, I])
    #print("Метрические коэффициенты:")
    #sym.pprint(G_)
    G = sym.Inverse(G_)
    #print("Обратная матрица метрических коэффициентов: ")
    #sym.pprint(G)
    H = sym.zeros(1, 3)
    for M in range(0, 3):
        #refine - упрощение выражений с использованием предположений. Предположения могут быть или в global_assumptions, или могут быть переданы напрямую при вызове функции.
        #powdenest - упрощение выражения со степенями, используя правило перемножения степеней при возведении степени в степень.
        H[M] = MySimplify(sym.sqrt(G_[M, M]))
    #print("Коэффициенты Ламе: ")
    #sym.pprint(H)
    Gamma = sym.MutableDenseNDimArray.zeros(3, 3, 3)
    for N in range(0, 3):
        for K in range(0, 3):
            for M in range(0, 3):
                Gamma[M, N, K] = sym.simplify(1/2*sum(G[M, S]*(sym.diff(G_[N, S], Q[K])+sym.diff(G_[K, S], Q[N])-sym.diff(G_[N, K], Q[S]))
                for S in range(3)))
    #print("Символы Кристоффеля второго рода:")
    #sym.pprint(Gamma)
    return Q, Rt, R_, H, Gamma

In [20]:
q, rt, r_, h, gamma = Reference_configuration(CYLINDRICAL)
e = sym.zeros(3, 3)
for m in range(3):
    for i in range(3):
        e[m, i] = 1/h[m]*r_[m, i]
Q, Rt, R_, H, Gamma = Actual_configuration(CYLINDRICAL)
E = sym.zeros(3, 3)
for M in range(3):
    for I in range(3):
        E[M, I] = 1/H[M]*R_[M, I]
A = sym.zeros(3, 3)
for M in range(3):
    for K in range(3):
        A[M, K] = dotprod(E[M, :], e[K, :])
A = A.subs([(R, P), (Phi, phi+Psi*z), (Z, k*z)])
Rt = Rt.subs([(R, P), (Phi, phi+Psi*z), (Z, k*z)])
C = Rt*A
for i in range(3):
    C[i] = C[i]/h[i]
C = MySimplifyMatrix(VectorGradient(C, q, gamma, h)*A.T)
print("Градиент деформации:")
display(C)

Градиент деформации:


⎡d                  ⎤
⎢──(P(r))    0     0⎥
⎢dr                 ⎥
⎢                   ⎥
⎢           P(r)    ⎥
⎢   0       ────   0⎥
⎢            r      ⎥
⎢                   ⎥
⎣   0      ψ⋅P(r)  k⎦

In [21]:
G = MySimplifyMatrix(C*C.T)
print("Мера деформации Коши:")
display(G)

Мера деформации Коши:


⎡          2                        ⎤
⎢⎛d       ⎞                         ⎥
⎢⎜──(P(r))⎟      0           0      ⎥
⎢⎝dr      ⎠                         ⎥
⎢                                   ⎥
⎢               2            2      ⎥
⎢              P (r)      ψ⋅P (r)   ⎥
⎢     0        ─────      ───────   ⎥
⎢                 2          r      ⎥
⎢                r                  ⎥
⎢                                   ⎥
⎢                2                  ⎥
⎢             ψ⋅P (r)   2    2  2   ⎥
⎢     0       ───────  k  + ψ ⋅P (r)⎥
⎣                r                  ⎦

In [22]:
print("Рассматриваем упрощённый вариант материала Блейтца и Ко.")
W1 = BlatzAndKoSimple()
display(sym.Eq(sym.Symbol('W'), W1))

Рассматриваем упрощённый вариант материала Блейтца и Ко.


      ⎛I₂       ____    ⎞
    μ⋅⎜── + 2⋅╲╱ I₃  - 5⎟
      ⎝I₃               ⎠
W = ─────────────────────
              2          

In [23]:
t = time.time()
PT = PiolaTensorSimple(G, C)
D = MySimplifyMatrix(2*dW(W1, G)*C)
DA = D*A
for m in range(3):
    for n in range(3):
        DA[m, n] = DA[m, n]/h[m]/h[n]
divD = TensorSecondRankDiv(DA, q, gamma, h)
divD = MySimplifyMatrix(divD*(A.T))
simpleTime = round(time.time()-t, 3) #Округление до миллисекунд.

In [24]:
print("Тензор Пиола (по выражению для упрощённого варианта материала Блейтца и Ко):")
display(PT)
print("Тензор Пиола (по основному выражению):")
display(D)
if PT == D:
    print("Оба тензора Пиола равны.")

Тензор Пиола (по выражению для упрощённого варианта материала Блейтца и Ко):


⎡k⋅μ⋅P(r)        μ                                                            
⎢──────── - ───────────                0                                      
⎢   r                 3                                                       
⎢           ⎛d       ⎞                                                        
⎢           ⎜──(P(r))⎟                                                        
⎢           ⎝dr      ⎠                                                        
⎢                                                                             
⎢                                           3      2  3                       
⎢                            d           μ⋅r    μ⋅ψ ⋅r              d         
⎢          0             k⋅μ⋅──(P(r)) - ───── - ───────  - μ⋅ψ⋅P(r)⋅──(P(r)) +
⎢                            dr          3       2                  dr        
⎢                                       P (r)   k ⋅P(r)                       
⎢                                                   

Тензор Пиола (по основному выражению):


⎡k⋅μ⋅P(r)        μ                                                            
⎢──────── - ───────────                0                                      
⎢   r                 3                                                       
⎢           ⎛d       ⎞                                                        
⎢           ⎜──(P(r))⎟                                                        
⎢           ⎝dr      ⎠                                                        
⎢                                                                             
⎢                                           3      2  3                       
⎢                            d           μ⋅r    μ⋅ψ ⋅r              d         
⎢          0             k⋅μ⋅──(P(r)) - ───── - ───────  - μ⋅ψ⋅P(r)⋅──(P(r)) +
⎢                            dr          3       2                  dr        
⎢                                       P (r)   k ⋅P(r)                       
⎢                                                   

Оба тензора Пиола равны.


In [25]:
print("Элемент [0, 0] тензора Пиола:")
display(D[0, 0])
print("Элемент [1, 1] тензора Пиола:")
display(D[1, 1])

Элемент [0, 0] тензора Пиола:


k⋅μ⋅P(r)        μ     
──────── - ───────────
   r                 3
           ⎛d       ⎞ 
           ⎜──(P(r))⎟ 
           ⎝dr      ⎠ 

Элемент [1, 1] тензора Пиола:


                   3      2  3
    d           μ⋅r    μ⋅ψ ⋅r 
k⋅μ⋅──(P(r)) - ───── - ───────
    dr          3       2     
               P (r)   k ⋅P(r)

In [26]:
print("Уравнение равновесия и граничное условие:")
display(sym.Eq(sym.diff(P, r, 2),
               sym.solve(divD[0], sym.diff(P, r, 2))[0]))
display(sym.Eq(D[0, 0], 0))
print("Время выполнения вычислений: "+str(simpleTime)+" секунд.")

Уравнение равновесия и граничное условие:


                           4           
               2 ⎛d       ⎞    d       
  2           r ⋅⎜──(P(r))⎟    ──(P(r))
 d               ⎝dr      ⎠    dr      
───(P(r)) = - ────────────── + ────────
  2                 3            3⋅r   
dr               3⋅P (r)               

k⋅μ⋅P(r)        μ         
──────── - ─────────── = 0
   r                 3    
           ⎛d       ⎞     
           ⎜──(P(r))⎟     
           ⎝dr      ⎠     

Время выполнения вычислений: 5.62 секунд.


In [27]:
print("Рассматриваем гипотетический вариант материала Блейтца и Ко.")
W2 = BlatzAndKoHypothetical()
display(sym.Eq(sym.Symbol('W'), W2))

Рассматриваем гипотетический вариант материала Блейтца и Ко.


      ⎛                -α⎞
      ⎜         -1 + I₃  ⎟
    μ⋅⎜I₁ - 3 + ─────────⎟
      ⎝             α    ⎠
W = ──────────────────────
              2           

In [28]:
t = time.time()
D = MySimplifyMatrix(2*dW(W2, G)*C)
DA = D*A
for m in range(3):
    for n in range(3):
        DA[m, n] = DA[m, n]/h[m]/h[n]
divD = TensorSecondRankDiv(DA, q, gamma, h)
divD = MySimplifyMatrix(divD*(A.T))
hypotheticalTime = round(time.time()-t, 3) #Округление до миллисекунд.

In [29]:
print("Уравнение равновесия и граничное условие:")
display(sym.Eq(sym.diff(P, r, 2), sym.solve(divD[0], sym.diff(P, r, 2))[0]))
display(sym.Eq(D[0, 0], 0))
print("Время выполнения вычислений: "+str(hypotheticalTime)+" секунд.")

Уравнение равновесия и граничное условие:


                                                                              
                                                                2             
                 2⋅α + 1      d               2⋅α + 2 ⎛d       ⎞     2⋅α  2⋅α 
  2         2⋅α⋅r       ⋅P(r)⋅──(P(r)) - 2⋅α⋅r       ⋅⎜──(P(r))⎟  + k   ⋅P    
 d                            dr                      ⎝dr      ⎠              
───(P(r)) = ──────────────────────────────────────────────────────────────────
  2                                                                           
dr                                                                            
                                                                              
                                                                              
                                                                              

                                                 α                            
                 2⋅α + 2         ⎛             2⎞  

                               -α                 
         ⎛                   2⎞                   
     2⋅α ⎜ 2  2    ⎛d       ⎞ ⎟                   
  μ⋅r   ⋅⎜k ⋅P (r)⋅⎜──(P(r))⎟ ⎟                   
         ⎝         ⎝dr      ⎠ ⎠       d           
- ─────────────────────────────── + μ⋅──(P(r)) = 0
              d                       dr          
              ──(P(r))                            
              dr                                  

Время выполнения вычислений: 8.881 секунд.


In [30]:
print("Рассматриваем полное представление материала Блейтца и Ко.")
W3 = BlatzAndKoFull()
display(sym.Eq(sym.Symbol('W'), W3))

Рассматриваем полное представление материала Блейтца и Ко.


        ⎛                -α⎞             ⎛           α    ⎞
        ⎜         -1 + I₃  ⎟             ⎜I₂       I₃  - 1⎟
    β⋅μ⋅⎜I₁ - 3 + ─────────⎟   μ⋅(1 - β)⋅⎜── - 3 + ───────⎟
        ⎝             α    ⎠             ⎝I₃          α   ⎠
W = ──────────────────────── + ────────────────────────────
               2                            2              

In [31]:
t = time.time()
D = MySimplifyMatrix(2*dW(W3, G)*C)
DA = D*A
for m in range(3):
    for n in range(3):
        DA[m, n] = DA[m, n]/h[m]/h[n]
divD = TensorSecondRankDiv(DA, q, gamma, h)
divD = MySimplifyMatrix(divD*(A.T))
fullTime = round(time.time()-t, 3) #Округление до миллисекунд.

In [32]:
print("Уравнение равновесия и граничное условие:")
display(sym.Eq(sym.diff(P, r, 2), sym.solve(divD[0], sym.diff(P, r, 2))[0]))
display(sym.Eq(D[0, 0], 0))
print("Время выполнения вычислений: "+str(fullTime)+" секунд.")

Уравнение равновесия и граничное условие:


            ⎛                               α                                 
            ⎜                  ⎛          2⎞                                  
            ⎜       4  4⋅α + 1 ⎜⎛d       ⎞ ⎟   3                              
            ⎜2⋅α⋅β⋅k ⋅r       ⋅⎜⎜──(P(r))⎟ ⎟ ⋅P (r)                     ⎛     
            ⎜                  ⎝⎝dr      ⎠ ⎠                 4  4⋅α + 2 ⎜⎛d   
            ⎜────────────────────────────────────── - 2⋅α⋅β⋅k ⋅r       ⋅⎜⎜──(P
            ⎜               d                                           ⎝⎝dr  
            ⎜               ──(P(r))                                          
  2         ⎜               dr                                                
 d          ⎝                                                                 
───(P(r)) = ──────────────────────────────────────────────────────────────────
  2                                                                           
dr                                                  

                  ⎛                                       ⎛                   
                  ⎜                        4⋅α + 2        ⎜                 2 
             -2⋅α ⎜          4⋅α ⎛d       ⎞           2⋅α ⎜   2⋅α ⎛d       ⎞  
-μ⋅(k⋅r⋅P(r))    ⋅⎜β⋅(k⋅P(r))   ⋅⎜──(P(r))⎟        + r   ⋅⎜β⋅r   ⋅⎜──(P(r))⎟  
                  ⎝              ⎝dr      ⎠               ⎝       ⎝dr      ⎠  

                          α                                       α           
    ⎛                   2⎞            4     ⎛                   2⎞    ⎛       
    ⎜ 2  2    ⎛d       ⎞ ⎟  ⎛d       ⎞      ⎜ 2  2    ⎛d       ⎞ ⎟    ⎜ 2  2  
- β⋅⎜k ⋅P (r)⋅⎜──(P(r))⎟ ⎟ ⋅⎜──(P(r))⎟  - β⋅⎜k ⋅P (r)⋅⎜──(P(r))⎟ ⎟  + ⎜k ⋅P (r
    ⎝         ⎝dr      ⎠ ⎠  ⎝dr      ⎠      ⎝         ⎝dr      ⎠ ⎠    ⎝       

              α⎞                                ⎞                       
            2⎞ ⎟                         4⋅α + 2⎟           -2⋅α - 3    
  ⎛d       ⎞ ⎟ ⎟           4⋅α ⎛d       ⎞       ⎟ ⎛d       ⎞  

Время выполнения вычислений: 74.709 секунд.


In [33]:
print("Рассматриваем тело Синьорини.")
W4 = SignoriniBody()
display(sym.Eq(sym.Symbol('W'), W4))

Рассматриваем тело Синьорини.


           ⎛  ⎛   ⎛I₂    ⎞              ⎞                          ⎞
           ⎜  ⎜I₂⋅⎜── + 3⎟          ____⎟                         2⎟
           ⎜  ⎜   ⎝I₃    ⎠         ╱ 1  ⎟               ⎛ I₂     ⎞ ⎟
           ⎜μ⋅⎜─────────── - 18⋅  ╱  ── ⎟   (9⋅λ + 5⋅μ)⋅⎜──── - 1⎟ ⎟
      ____ ⎜  ⎝     I₃          ╲╱   I₃ ⎠               ⎝3⋅I₃    ⎠ ⎟
W = ╲╱ I₃ ⋅⎜───────────────────────────── + ───────────────────────⎟
           ⎝              18                           8           ⎠

In [34]:
t = time.time()
D = MySimplifyMatrix(2*dW(W4, G)*C)
DA = D*A
for m in range(3):
    for n in range(3):
        DA[m, n] = DA[m, n]/h[m]/h[n]
divD = TensorSecondRankDiv(DA, q, gamma, h)
divD = MySimplifyMatrix(divD*(A.T))
signoriniTime = round(time.time()-t, 3) #Округление до миллисекунд.

In [35]:
print("Уравнение равновесия и граничное условие:")
display(sym.Eq(sym.diff(P, r, 2), sym.solve(divD[0], sym.diff(P, r, 2))[0]))
display(sym.Eq(D[0, 0], 0))
print("Время выполнения вычислений: "+str(signoriniTime)+" секунд.")

Уравнение равновесия и граничное условие:


            ⎛                    3                          2                 
            ⎜   4    3 ⎛d       ⎞     4    2      ⎛d       ⎞       4      2   
  2         ⎜- k ⋅λ⋅r ⋅⎜──(P(r))⎟  - k ⋅λ⋅r ⋅P(r)⋅⎜──(P(r))⎟  + 3⋅k ⋅λ⋅r⋅P (r)
 d          ⎝          ⎝dr      ⎠                 ⎝dr      ⎠                  
───(P(r)) = ──────────────────────────────────────────────────────────────────
  2                                                                           
dr                                                                            
                                                                              
                                                                              

           3                                                  2               
 ⎛d       ⎞     4      2    d             4    3    ⎛d       ⎞     4    3     
⋅⎜──(P(r))⎟  + k ⋅λ⋅r⋅P (r)⋅──(P(r)) - 3⋅k ⋅λ⋅P (r)⋅⎜──(P(r))⎟  + k ⋅λ⋅P (r) -
 ⎝dr      ⎠                 dr                     

                        4                                   2                 
 4  4         ⎛d       ⎞       4  2          2    ⎛d       ⎞       4  2       
k ⋅r ⋅(λ + μ)⋅⎜──(P(r))⎟  - 2⋅k ⋅r ⋅(λ + μ)⋅P (r)⋅⎜──(P(r))⎟  - 2⋅k ⋅r ⋅(3⋅λ +
              ⎝dr      ⎠                          ⎝dr      ⎠                  
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              
                                                                              

                    4                                                       2 
     2    ⎛d       ⎞       4          4         4            4    ⎛d       ⎞  
 μ)⋅P (r)⋅⎜──(P(r))⎟  - 3⋅k ⋅(λ + μ)⋅P (r) + 2⋅k ⋅(3⋅λ + μ)⋅P (r)⋅⎜──(P(r))⎟  
          ⎝dr      ⎠                               

Время выполнения вычислений: 37.204 секунд.


In [36]:
print("Рассматриваем материал Мурнагана.")
W5 = MurnaghanMaterial()
display(sym.Eq(sym.Symbol('W'), W5))

Рассматриваем материал Мурнагана.


      3 ⎛l   m⎞     2 ⎛  3⋅l   λ        ⎞                ⎛9⋅l               n⎞
    I₁ ⋅⎜─ + ─⎟   I₁ ⋅⎜- ─── + ─ - m + μ⎟             I₁⋅⎜─── - 3⋅λ - 2⋅μ + ─⎟
        ⎝6   3⎠       ⎝   2    2        ⎠   I₁⋅I₂⋅m      ⎝ 2                2⎠
W = ─────────── + ─────────────────────── - ─────── + ────────────────────────
         4                   4                 4                 4            

      ⎛            n⎞             
   I₂⋅⎜3⋅m - 2⋅μ - ─⎟             
      ⎝            2⎠   n⋅(I₃ - 1)
 + ────────────────── + ──────────
           4                8     

In [37]:
t = time.time()
D = MySimplifyMatrix(2*dW(W5, G)*C)
DA = D*A
for m in range(3):
    for n in range(3):
        DA[m, n] = DA[m, n]/h[m]/h[n]
divD = TensorSecondRankDiv(DA, q, gamma, h)
divD = MySimplifyMatrix(divD*(A.T))
murnaghanTime = round(time.time()-t, 3) #Округление до миллисекунд.

In [38]:
print("Уравнение равновесия и граничное условие:")
display(sym.Eq(sym.diff(P, r, 2), sym.solve(divD[0], sym.diff(P, r, 2))[0]))
display(sym.Eq(D[0, 0], 0))
print("Время выполнения вычислений: "+str(murnaghanTime)+" секунд.")

Уравнение равновесия и граничное условие:


                                                                              
             4    2  6         4    5 d           4    4           4    2  6  
  2         k ⋅l⋅ψ ⋅r ⋅P(r) - k ⋅l⋅r ⋅──(P(r)) + k ⋅l⋅r ⋅P(r) + 2⋅k ⋅m⋅ψ ⋅r ⋅P
 d                                    dr                                      
───(P(r)) = ──────────────────────────────────────────────────────────────────
  2                                                                           
dr                                                                            
                                                                              
                                                                              

                                                       2                      
         2    4  6  3         2    2  6      ⎛d       ⎞       2    2  6       
(r) + 2⋅k ⋅l⋅ψ ⋅r ⋅P (r) - 2⋅k ⋅l⋅ψ ⋅r ⋅P(r)⋅⎜──(P(r))⎟  - 6⋅k ⋅l⋅ψ ⋅r ⋅P(r) -
                                             ⎝dr   

⎛  ⎛                                                                          
⎜  ⎜             4      ⎛                             2⎞                      
⎜  ⎜ 4 ⎛d       ⎞     2 ⎜ 2  4       2 ⎛ 2    2  2   ⎞ ⎟   ⎛ 2  2    ⎞  4     
⎜m⋅⎜r ⋅⎜──(P(r))⎟  + r ⋅⎝ψ ⋅P (r) + r ⋅⎝k  + ψ ⋅P (r)⎠ ⎠ + ⎝ψ ⋅r  + 1⎠⋅P (r) -
⎝  ⎝   ⎝dr      ⎠                                                             
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                           2⎞                                 
 ⎛   ⎛                          2⎞        ⎞ ⎟                                 
 ⎜ 2 ⎜ 2    2  2      ⎛d       ⎞ ⎟    2   ⎟ ⎟    4                          2 
 ⎜r ⋅⎜k  + ψ ⋅P (r) + ⎜──(P(r))⎟ ⎟ + P (r)⎟ ⎟ + r ⋅(9⋅l - 6⋅λ - 4⋅μ + n) + r ⋅
 ⎝   ⎝                ⎝dr      ⎠ ⎠        ⎠ ⎠      

Время выполнения вычислений: 35.623 секунд.


In [39]:
timeData = {'Функция энергии':
            ["Материал Блейтца и Ко (упрощённая модель)",
             "Материал Блейтца и Ко (гипотетический вариант)",
             "Материал Блейтца и Ко (полная трёхконстантная модель)",
             "Тело Синьорини",
             "Материал Мурнагана"],
            "Время, c": [simpleTime, hypotheticalTime, fullTime, signoriniTime, murnaghanTime]}
df = pd.DataFrame(data=timeData)
display(df)
print("Процессор, с помощью которого выполнялись вычисления:")
display(cpuinfo.get_cpu_info()['brand_raw'])

,Функция энергии,"Время, c"
0,Материал Блейтца и Ко (упрощённая модель),5.620
1,Материал Блейтца и Ко (гипотетический вариант),8.881
2,Материал Блейтца и Ко (полная трёхконстантная ...,74.709
3,Тело Синьорини,37.204
4,Материал Мурнагана,35.623


Процессор, с помощью которого выполнялись вычисления:


'Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz'